# Emotion_detection

**Import necessary packages**



In [ ]:
from tensorflow.python import keras
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from google.colab.patches import cv2_imshow # for colab
from keras.preprocessing import image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL

# Load input video for prediction using opencv VideoCapture method

In [ ]:

height = 768
width = 576

cap = cv2.VideoCapture('/content/drive/MyDrive/checkpoint/Emotion.mp4')
#cap = cv2.VideoWriter('output.avi', cv.VideoWriter_fourcc(*'XVID'), 25, (320,240))

cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)






# Install necessary pacakge to write output video file

In [ ]:
! add-apt-repository -y ppa:savoury1/ffmpeg4
! apt-get -qq install -y ffmpeg

Repository: 'deb https://ppa.launchpadcontent.net/savoury1/ffmpeg4/ubuntu/ jammy main'
Description:
FFmpeg 4.4.4 builds (& associated multimedia packages) for Xenial & newer.

*** Anyone interested in full builds of FFmpeg 4.4.x including all "bells and whistles" needs to have donated, after which access to the new private PPA can be requested. See my Launchpad page linked below for details. ***

*** Please always see https://launchpad.net/~savoury1 for general updates about this Launchpad site before contacting me or reporting any bugs! ***

*** Big thanks to all those who have donated to support this project, you are very directly helping to keep it alive! To all who have not donated: please do so if you can afford it, this project depends on donations. ***

If software at this site is useful to you then please consider a donation:

*** Donations: https://paypal.me/Savoury1 & https://ko-fi.com/Savoury1 ***
*** Also https://patreon.com/Savoury1 & https://liberapay.com/Savoury1 ***

Up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Set class labels, load opencv Haascascade frontalface classifier and Human emotion detection model that i have trained.

In [ ]:
class_labels=['Angry', 'disgusted', 'fearful', 'Happy','Neutral','Sad','Surprise']

face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

classifier = load_model("/content/drive/MyDrive/checkpoint/model.keras")

# Get height & width of output file and create object of videowriter to save predictions in an output file.

In [ ]:
Height=int(cap.get(3))
Width=int(cap.get(4))
frame_size= (Height, Width)

fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
output = cv2.VideoWriter('/content/drive/MyDrive/Emotion_capture/emotion_detection.mp4', fourcc, 20, frame_size)

# Prediction:

Reading video input file using videocapture object.

Passing the video frame to Haarcascade classifier's detectMultiScale function to detect faces.

Then converting frame from BGR to RGB mode as opencv reads image in BGR mode.

Looping over detected faces to detect emotions.

In [ ]:
while cap.isOpened():
  ret, img = cap.read()
  #print(np.shape(img))


  if np.shape(img) == (): # If video frame is empty then break the loop
    break



  faces = face_cascade.detectMultiScale(img, scaleFactor = 1.1, minNeighbors = 6, minSize =(60, 60),flags= cv2.CASCADE_SCALE_IMAGE)

  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  #cv2_imshow(img)


  for (x,y,w,h) in faces: # Iterrate over detected faces from video frame
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) # Drwa rectangle over detcted faces
      #cv2_imshow(rgb)
      roi_rgb=rgb[y:y+h,x:x+w] # Selecting 'Region of Interest' from image
      roi_rgb=cv2.resize(roi_rgb,(255,255),interpolation=cv2.INTER_NEAREST) # resize video frame to desired input size of model
      if np.sum([roi_rgb])!=0:
        roi=img_to_array(roi_rgb) # converting image to array
        #roi=roi.astype('float')/255.0
        roi=np.expand_dims(roi,axis=0)# expand dimension of input array
        preds=classifier.predict(roi)[0]  # predict human emotion
        label=class_labels[preds.argmax()] # assign predicted label
        label_position=(x,y)
        cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
      else:
        cv2.putText(img,'No Face Found',(20,20),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)


      cv2_imshow(img)
      output.write(img)

      if cv2.waitKey(40) == 27:
          break

cv2.destroyAllWindows()
cap.release()

